# Simple Demo about MongoDB FDW of PostgreSQL
## MongoDB connection in Python

In [17]:
import os
import numpy as np
from pymongo import MongoClient
import datetime
from random import randrange
from random import randint
client =  MongoClient("mongodb://mongo:27017")
db = client.primer
ds = db.dataset
result = ds.delete_many({})
ds.insert_many([{'id': i,
                 'price':20000 + np.random.normal(0, 0.5, 1)[0],
                 'coord_id':randint(1,1000),
                 'time': datetime.datetime.strptime(str(datetime.date(randint(2000,2017), randint(1,12),1)), "%Y-%m-%d")
                } for i in range(1,10**4)]).inserted_ids
ds.count()

9999

## PostgreSQL connection in Python

In [ ]:
import psycopg2
conn = psycopg2.connect(database="gis", user="mongo", password="mongotest", host="db", port="5432")
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall() )
conn.close() 

In [19]:
import pprint
pprint.pprint(ds.find_one())

{u'_id': ObjectId('593d00cad592b5004cf8ae07'),
 u'coord_id': 860,
 u'id': 1,
 u'price': 20000.522469014984,
 u'time': datetime.datetime(2010, 10, 1, 0, 0)}
